# Web scraping:

In this notebook, we will perform web-scraping on revenues and budgets. 

***First, we query in sparql wikidata identifiers using freebase ids.***

***
    SELECT ?movie ?movieLabel ?freebaseID WHERE {  
    
    ?movie p:P646 [ps:P646 ?freebaseID].  
    ?movie wdt:P31 wd:Q11424.
    ?movie wdt:P495 wd:Q30.  
    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }  
    
    }
    
     
***
  

This query selects all movies from US that have a freebase id and outputs the file query_american_movies.

In [15]:
'''
File name : web_scraping.ipynb
Author : Group MrJulia
Date created : '18/11/2022'
Python version : 3.6
'''
import pandas as pd 
import pywikibot

cmu_folder = './Data/Cmu_data/'
scraping_folder='./Data/Scraping/'
queries_folder='./Data/Queries/'

wikis= pd.read_csv(queries_folder+"query_american_movies.csv")
wikis.head()

,movie,movieLabel,freebaseID
0,http://www.wikidata.org/entity/Q33109,Be Like Others,/m/03qlwpd
1,http://www.wikidata.org/entity/Q36055,Grunts,/m/04cvqhb
2,http://www.wikidata.org/entity/Q39975,Dazed and Confused,/m/02j69w
3,http://www.wikidata.org/entity/Q40115,Chasing Amy,/m/0gd92
4,http://www.wikidata.org/entity/Q44085,Private School,/m/0glhwy


In [16]:
#keep only the movie identifier in wikidata
wikis['movie'] = wikis['movie'].apply(lambda x : x.split('/')[-1])
print('The number of movies we will try to scrape :'+str(len(wikis)))
wikis.head()

The number of movies we will try to scrape :38377


,movie,movieLabel,freebaseID
0,Q33109,Be Like Others,/m/03qlwpd
1,Q36055,Grunts,/m/04cvqhb
2,Q39975,Dazed and Confused,/m/02j69w
3,Q40115,Chasing Amy,/m/0gd92
4,Q44085,Private School,/m/0glhwy


We query each identifier and get the properties of our interest from the claims

In [17]:
def wiki_search(wiki_id) :
    '''
    Given its wikidata id, searches for the revenue and budget of a movie if available
    param: wiki_id : item identifier in wikidata
    returns : revenue and budget as a tuple, 0 for each unavailable property
    '''
    #revenue and budget properties identifiers
    properties={'P2142':0,'P2130':0}
    site = pywikibot.Site("wikidata", "wikidata")
    repo = site.data_repository()
    movie = pywikibot.ItemPage(repo, wiki_id).get()
    for interest in properties :
        if interest in movie['claims'] :
            for field in movie['claims'][interest] :
                properties[interest]=field.getTarget().amount
    return properties.values()

In [18]:
#perform the search on all movies identifiers
wikis['result']=wikis['movie'].apply(wiki_search)
wikis.head()

,movie,movieLabel,freebaseID,result
0,Q33109,Be Like Others,/m/03qlwpd,"(0, 0)"
1,Q36055,Grunts,/m/04cvqhb,"(0, 0)"
2,Q39975,Dazed and Confused,/m/02j69w,"(0, 0)"
3,Q40115,Chasing Amy,/m/0gd92,"(12021272, 250000)"
4,Q44085,Private School,/m/0glhwy,"(0, 0)"


We can now create separate columns for our properties

In [19]:
vals=pd.DataFrame(wikis['result'].tolist(),index=wikis.index)
wikis['revenue']= vals[0]
wikis['budget']= vals[1]
wikis=wikis.drop(columns=['result'])
wikis.head()

,movie,movieLabel,freebaseID,revenue,budget
0,Q33109,Be Like Others,/m/03qlwpd,0,0
1,Q36055,Grunts,/m/04cvqhb,0,0
2,Q39975,Dazed and Confused,/m/02j69w,0,0
3,Q40115,Chasing Amy,/m/0gd92,12021272,250000
4,Q44085,Private School,/m/0glhwy,0,0


We will consider only valid query responses which means in our case getting non zero revenue and budget

In [20]:
valid = wikis[wikis['revenue']!=0]
valid = valid[valid['budget']!=0]
valid.reset_index(drop=True, inplace=True)
print('The number of valid scraped movie properties is: '+str(len(valid)))
valid.head()

The number of valid scraped movie properties is: 1233


,movie,movieLabel,freebaseID,revenue,budget
0,Q40115,Chasing Amy,/m/0gd92,12021272,250000
1,Q45384,Carrie,/m/0jwzklz,84790678,30000000
2,Q39999,Mallrats,/m/01jc1h,2122561,6000000
3,Q46551,The Prestige,/m/084302,109676311,40000000
4,Q39970,Buffy the Vampire Slayer,/m/01cbx,16624456,7000000


In [7]:
#write the file to our folder
valid.to_csv(scraping_folder+'wikis_scrape.csv',index=False)

***On the other hand, we will perform scraping from tmdb database using the tmdbsimple api***

We first have to filter the dataframe from our initial data that consists only on film from the US, the same way we did during
preprocessing

In [8]:
colnames=["Wikipedia movie ID","Freebase movie ID","Movie name","Movie release date","Movie box office revenue","Movie runtime","Movie languages","Movie countries","Movie genres"]
df_movies=pd.read_csv(cmu_folder+'/movie.metadata.tsv', sep="\t", names = colnames)
df_movies.head()

,Wikipedia movie ID,Freebase movie ID,Movie name,Movie release date,Movie box office revenue,Movie runtime,Movie languages,Movie countries,Movie genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"


In [9]:
def extract_values(x) :
    '''
    Extracts the values from a dictionary string
    param: x : dictionary string to be processed
    returns : a list of the dictionary values
    '''
    #split keys and values
    b = x.split(':')
    #extract strings
    b = sum([xb.split('"') for xb in b],[])
    #reduce white spaces of any length to empty strings
    b = [xb.strip() for xb in b]
    #remove empty strings, only the values start with alphanumeric characters
    b = [xb for xb in b if(xb!='' and xb[0].isalnum())]
    return b

In [10]:
#filter only the US movies
df_movies['Movie countries']=df_movies['Movie countries'].apply(extract_values)
df_movies=df_movies.explode('Movie countries')
df_movies=df_movies[df_movies['Movie countries']=='United States of America']
df_movies.reset_index(drop=True, inplace=True)
print("The number of movies to query is: "+str(len(df_movies)))
df_movies.head()

The number of movies to query is: 34408


,Wikipedia movie ID,Freebase movie ID,Movie name,Movie release date,Movie box office revenue,Movie runtime,Movie languages,Movie countries,Movie genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}",United States of America,"{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}",United States of America,"{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,13696889,/m/03cfc81,The Gangsters,1913-05-29,NaN,35.0,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""Eng...",United States of America,"{""/m/02hmvc"": ""Short Film"", ""/m/06ppq"": ""Silen..."
3,10408933,/m/02qc0j7,Alexander's Ragtime Band,1938-08-16,3600000.0,106.0,"{""/m/02h40lc"": ""English Language""}",United States of America,"{""/m/04t36"": ""Musical"", ""/m/01z4y"": ""Comedy"", ..."
4,175026,/m/017n1p,Sarah and Son,1930,NaN,86.0,"{""/m/02h40lc"": ""English Language""}",United States of America,"{""/m/07s9rl0"": ""Drama"", ""/m/01g6gs"": ""Black-an..."


To perform our query, we need the movie name, its release date to check if it is indeed the movie we are looking for (some movies share the same name) and the freebase id to be able to merge back later on.  
The query needs the movie name to be in lower case and to consist of its joined words with the + character.

In [11]:
df_movies['Movie name'] = df_movies.apply(lambda x : '+'.join([name.lower() for name in x['Movie name'].split(" ")]),axis=1)
df_movies['Movie release date']=df_movies['Movie release date'].astype('str').apply(lambda x : x.split('-')[0])
query_movies= df_movies[['Freebase movie ID', 'Movie name', 'Movie release date']]
query_movies.head()

,Freebase movie ID,Movie name,Movie release date
0,/m/03vyhn,ghosts+of+mars,2001
1,/m/08yl5d,getting+away+with+murder:+the+jonbenét+ramsey+...,2000
2,/m/03cfc81,the+gangsters,1913
3,/m/02qc0j7,alexander's+ragtime+band,1938
4,/m/017n1p,sarah+and+son,1930


In [12]:
import tmdbsimple as tmdb
import requests
import locale

tmdb.API_KEY = 'API KEY HERE'
local_key = '8341de77e3a24593226752d83720f88b'

In [13]:
def search(response,name,freebase_id,year) :
    '''
    Looks for a movie's revenue and budget after the website response if available
    param: response : website response to the query
           name, freebase_id, year: fields of the movie to look for
    returns : a list of freebaseid, revenue and budget if the movie was found -1 otherwise
    '''
    found=0
    for film in response['results'] :
        film_revenue = requests.get('https://api.themoviedb.org/3/movie/'+ str(film['id']) +'?api_key='+ local_key).json()
        if(film_revenue['release_date'].split('-')[0]==year) :
            found=1
            break
    return [freebase_id,film_revenue['revenue'],film_revenue['budget']] if found else -1

In [14]:
def scrape(query_movies) :
    '''
    Scrapes for revenue and budget over a dataframe of movies
    param: query_movies : dataframe of movies, need to contain movie name, release year and freebase id.
    returns : a dataframe that contains properties of the available movies
    '''
    data = []
    for i in range(len(query_movies)) :
        name = query_movies['Movie name'].iloc[i]
        year = query_movies['Movie release date'].iloc[i]
        freebase_id =  query_movies['Freebase movie ID'].iloc[i] 
        url = 'https://api.themoviedb.org/3/search/movie?api_key='+local_key+'&query='+name
        response = requests.get(url).json()
        if 'results' in response :
            #if the query yields possible results, search among them
            res = search(response,name,freebase_id,year)
        if res != -1 : data.append(res)
    d=pd.DataFrame(data)
    return d

***The query is time consuming. To proceed, we decide to use chunks of 10 000 queries, which means in our case 4 functions calls.***

In [15]:
chunk=10000
a=scrape(query_movies[:chunk])
b=scrape(query_movies[chunk:chunk*2])
c=scrape(query_movies[chunk*2:chunk*3])
d=scrape(query_movies[chunk*3:])

In [16]:
tmdb_data= pd.concat([a,b,c,d])
tmdb_data= tmdb_data.rename(columns={0:'name',1:'revenue',2:'budget'})
tmdb_data.reset_index(drop=True, inplace=True)
tmdb_data.head()

,name,revenue,budget
0,/m/03vyhn,14010832,28000000
1,/m/08yl5d,0,0
2,/m/03cfc81,0,0
3,/m/02qc0j7,4000000,2000000
4,/m/017n1p,0,0


Finally, we consider again only valid query responses.

In [17]:
valid=tmdb_data[tmdb_data['revenue']!=0]
valid=valid[valid['budget']!=0]
valid.reset_index(drop=True, inplace=True)
print('The number of valid scraped movie properties is: '+str(len(valid)))
valid.head()

The number of valid scraped movie properties is: 4832


,name,revenue,budget
0,/m/03vyhn,14010832,28000000
1,/m/02qc0j7,4000000,2000000
2,/m/0kcn7,103082380,6000000
3,/m/014k4y,29381649,16000000
4,/m/0b6kc_5,6718,4000000


In [18]:
#write the file to our folder
valid.to_csv(scraping_folder+'tmdb_scrape.csv',index=False)